In [1]:
import nltk
from functools import reduce
import pandas as pd
import numpy as np
from collections import Counter

nltk.download('punkt')

data = pd.read_csv("estadao_news.csv")

[nltk_data] Downloading package punkt to /home/gustavo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data['articles'] = data['titulo']  + ' ' + data['conteudo']

In [3]:
tokenize = lambda row: nltk.word_tokenize(row)

In [4]:
data['tokens'] = data['articles'].apply(tokenize)

In [5]:
counter = lambda row: Counter(row)

In [11]:
data['tokens'] = data['tokens'].apply(counter)

In [23]:
counter = Counter()
for row_tokens in data['tokens']:
    counter += row_tokens